## Byte degree 미니 프로젝트
 - 숫자 퍼즐 게임 완성하기
 - 아래와 같이 숫자 퍼즐을 만들고 숫자를 이동시켜 순서대로 맞추는 게임
 - ![퍼즐이미지](https://i.stack.imgur.com/0B14h.png)
 - [이미지 출처](https://math.stackexchange.com/questions/635188/what-is-the-parity-of-permutation-in-the-15-puzzle)

In [9]:
import random

### 게임 로직 구현하기
 1. 퍼즐 생성하기
 2. 퍼즐 랜덤하게 섞기
 3. 퍼즐 출력
 4. 사용자 입력(움직일 숫자 입력 받기)
 5. 퍼즐 완성 확인하기
   - 완성? 완료 메시지와 함께 종료
   - 미완성? 3번으로 이동

#### 퍼즐 생성하기
 - 2차원 리스트 형태로 생성
 - 퍼즐의 크기(size)를 파라미터로 받아, 동적으로 size*size의 리스트로 생성
 - 퍼즐이 생성되면 1부터 차례대로 행방향으로 숫자를 나열
   - 사이즈가 3인 경우의 생성 예
   -  [[1, 2, 3],
      [4, 5, 6],
      [7, 8, 9]]
 - 퍼즐의 가장 마지막 아이템(마지막 행의 마지막 열 아이템)은 '' 빈문자열로 처리
   - 이유는? 숫자퍼즐의 목표는 빈공간을 이용해 각 이동하고자 하는 숫자를 빈공간으로 움직여 숫자들을 순서대로 다시 맞추는 것이 목적이므로, 빈공간을 표현하기 위한 방법으로 빈문자열을 사용

* pure python 버젼

In [10]:
# 밑줄 부분을 채워봅니다.


def initiate_puzzle(size):
    
    '''
    파라미터
     size: 퍼즐의 크기
    리턴
     생성된 퍼즐 리스트
    '''
    
    puzzle = []
    for i in range(size):
        row = []
        for j in range(size):
            row.append(i*size+1+j)
        puzzle.append(row)

    puzzle[-1][-1] = ''
    return puzzle

>- 리뷰
   - i와 j를 적절하게 잘 배치하셨습니다.
   - 또 j 값과 함께 +1을 같이 잘 주셨구요.
     - 아시다시피 +1이 없다면 j는 0부터 시작되기에 중복되는 값이 생길테니까요

* numpy 버젼

In [11]:
# 밑줄 부분을 채워봅니다.
import numpy as np
def initiate_puzzle(size):
    
    '''
    파라미터
     size: 퍼즐의 크기
    리턴
     생성된 퍼즐 리스트
    '''
    
    puzzle = np.arange(1,size*size+1).reshape(size,size)
    puzzle = puzzle.tolist() 
    puzzle[-1][-1] = ''
    return puzzle

---
>- 리뷰
   - 빈칸에 알맞게 잘 넣으셨습니다.
   - 직관적으로 size X size 라고도 적어주셨네요!
   - numpy version의 경우에도 그 위의 함수와 같은 결과가 나와야합니다.
   - numpy.arnage에서 size*size + 1을 잘 넣어주셨네요!
   - +1을 하지 않으면 size*size - 1만큼 생성되겠지요
     - 게다가 1부터 시작하도록 했으니 size * size -2만큼 생성되겠네요!

In [12]:
puzzle = initiate_puzzle(4)

#### 퍼즐 출력하기
 - 생성된 퍼즐(puzzle)을 파라미터로 받아 화면에 출력
 - 이때, 퍼즐은 2차원 형태이므로 2중 loop을 이용

In [13]:
# pass를 삭제하고 아래에 이중 for 문을 사용해 코드를 작성해봅니다.


def show_puzzle(puzzle):
    
    '''
    파라미터
     puzzle: 퍼즐 
    리턴
     None
    '''
    for row in puzzle:
        for i in row:
            print(format(i,"3"),end=" ")
        print()
    
    

---
>- 리뷰
   - print function을 적절하게 사용하셔서 아래와 같이 보기 좋게 출력하셨네요.
   - print function의 end의 default 값은 new line (=\\n)입니다. 
   - 이미 아시다시피 따로 설정해주지 않으면 다음줄로 표시되고 말아버리지요
   - row에 대한 for 문이 끝나고 print 호출하신 것도 적절하게 잘하셨습니다
     - 그렇지 않았다면 같은 줄에 계속 나왔을테니까요
   - 여담이지만 위에 사용하신 format()을 사용하셔도 되지만 위와 같이 사용할 경우 ''.format() 과 같은 형태로 주로 사용합니다.
     - ex)
         '{:3}'.format(i)
   - python3에서는 f'' 만으로 표시할 수  있어요!
     - 예)
          f'{i:3}'

In [14]:
show_puzzle(puzzle)

  1   2   3   4 
  5   6   7   8 
  9  10  11  12 
 13  14  15     


#### 퍼즐 섞기(shuffling)
 - 생성할때부터 랜덤하게 숫자를 배열하지 않고, 완성된 상태에서 퍼즐을 섞어야 함
   - 이유? 랜덤하게 배열하는 경우, 퍼즐이 완성되지 못하는 경우의 수가 수학적으로 존재하기 때문
   - 퍼즐을 완성시킬 수 없는 예
   - [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 15, 14, '']]
   - [출처: 네이버 블로그](https://post.naver.com/viewer/postView.nhn?volumeNo=17980703&memberNo=16868720) 

In [15]:
# 밑줄 부분을 채워봅니다.
def get_index(puzzle, n):
    
    '''
    파라미터
      puzzle: 퍼즐
      n: 퍼즐 내에서 찾으려는 숫자 혹은 빈공간('') 값
    리턴
      퍼즐에서 해당 숫자나 빈공간을 찾았다면 해당 인덱스를 반환
      찾지 못했다면 None, None 반환
    '''
    
    for i in range(len(puzzle)):
        try:
            index = puzzle[i].index(n)
            return (i,index) if not index==None else (None,None)
        except:
            continue
    return (None,None)

---
>- 리뷰
   - 적절한 attribute를 잘 찾아써주셨네요!
   - list 내에서 원하는 item의 index를 찾고 싶을때는 list.index(item)을 사용합니다.
     - 꽤나 직관적이지요
   - .index를 사용했을 떄 원하는 값이 list 내부에 없으면 Error가 납니다.
   - 이 때문에 try except를 사용했구요
     - list에 원하는 값이 없을 경우 except로 넘어갈 것입니다.
   - try 안의 return에서 if not index == None else (None, None)을 붙여주신 이유가 index에서 못 찾았을때를 위해 넣으신거 같아요
   - 그 경우를 위해 try except를 사용한 것이니 참고해주세요!
     - try except를 뺴고 위의 return을 그대로 사용했어도 index에서 error가 나기 때문에 return으로 가기 전에 코드가 끝나버리니 유의해주세요!

In [16]:

def shuffle_puzzle(puzzle, shuffle_count):
    
    '''
    파라미터
     puzzle: 퍼즐
     shuffle_count: 섞을 횟수
    리턴
     None
    '''
    
    # 각각 섞을 때마다 빈공간을 기준으로 상하좌우의 방향으로 섞기 위해
    # 방향 리스트 생성
    # 순서대로 상 우 하 좌 
    dxs = [1, 0, -1,  0]
    dys = [0, 1,  0, -1]

    cnt = 0
    while cnt <= shuffle_count:
        rnd = random.randint(0, 3)
        dx = dxs[rnd]
        dy = dys[rnd]

        # 빈공간의 index 얻기
        i, j = get_index(puzzle, '')            
        ni = i + dx
        nj = j + dy

        # 새로 얻은 인덱스 확인(퍼즐 범위내에 포함되는지) 하여 숫자 이동
        if 0 <= ni < len(puzzle) and 0 <= nj < len(puzzle[0]):
            puzzle[ni][nj], puzzle[i][j] = puzzle[i][j], puzzle[ni][nj]
        
        cnt += 1

In [17]:
shuffle_puzzle(puzzle, 10)
show_puzzle(puzzle)

  1   2   3   4 
  5   6   7   8 
  9  10  11  12 
 13      14  15 


#### 퍼즐이 완성되었는지 확인하기
 - 퍼즐이 완성된 형태인지 확인
 - puzzle 퍼즐로 활용할 리스트, completed 완성된 형태의 퍼즐 리스트 
 - 완성되었다면 True, 아니라면 False 반환

In [18]:
# pass를 삭제하고 아래에 return 코드를 작성해봅니다.

def is_puzzle_completed(puzzle, completed):
    for i in range(len(puzzle)):
        for j in range(len(puzzle)):
            if not puzzle[i][j]==completed[i][j]:
                return False
    return True

---
>- 리뷰
   - 과정이 다소 복잡하게 표현하셨지만 잘 하셨습니다.
   - 위의 function은 puzzle과 completed가 동일한가를 체크하는 일을 합니다.
   - 하나하나의 요소를 비교해도 되지만
   - 단순하게 puzzle == completed로 해도 둘이 똑같은 가를 비교할 수 있어요!

#### 퍼즐 이동하기
 - 퍼즐 내의 숫자를 이동
 - 이때 이동이 가능한 경우는 해당 숫자가 빈공간 상하좌우에 위치한 경우에만 가능

In [19]:
def move_by_number(puzzle, n):
    # 숫자가 위치한 index
    i, j = get_index(puzzle, n)
    
    # index를 이용하여 숫자 이동
    move_by_index(puzzle, i, j)

In [20]:
# 밑줄 부분을 채워봅니다.
def move_by_index(puzzle, i, j):
    # 좌우위아래 한방향중 하나가 '' 값이라면 이동 가능
    for dx, dy in ((1, 0), (0, 1), (-1, 0), (0, -1)):
        new_i = i + dx
        new_j = j + dy

        # boundary 체크(갈 수 없는 곳이면 패스)
        if (new_i>=len(puzzle)or new_i<0) or(new_j>=len(puzzle)or new_j<0) or (not puzzle[new_i][new_j]==''):
            continue

        # 옆에 빈 공간인 경우에는 퍼즐의 위치를 빈공간과 바꿈(swap)
        if puzzle[new_i][new_j] == '':
            puzzle[i][j],puzzle[new_i][new_j]=puzzle[new_i][new_j],puzzle[i][j]
            return 

---
>- 리뷰
   - 복잡한 조건문이었는데 아주 잘 써주셨습니다.
   - 위를 좀 더 단순히 하자면 갈 수 있는 곳이 아닌 경우 continue를 하면 더 직관적으로 보일 것 같아요.
     - if not (boundary 조건문)
   - 아시다시피 new_i, new_j는 puzzle의 가로 세로 길이를 벗어나면 안되어 len(puzzle)을 사용하고
   - 음수가 나올 가능성이 있기 때문에 0에 대한 조건도 붙어야합니다.
   - 마지막으로 swap에서도 아주 잘 써주셨습니다.
   - python의 장점으로 위와 같이 한줄로 swap을 진행할 수 있습니다.

#### 사용자 프롬프트 입력
 - 게임의 진행을 위해 동적으로 키보드 입력을 받을 필요가 있음
   - 퍼즐의 크기, 이동할 수 지정 
 - 이를 위해 input 함수 사용
   - 원하는 값 입력후, Enter

In [21]:
value = input('입력하세요')
print(value)

입력하세요4
4


* 입력받은 값을 숫자형태로 변경

In [22]:
value = int(input('숫자를 입력하세요'))
print(value)

숫자를 입력하세요4
4


#### 퍼즐, 퍼즐 완성본 생성 및 셔플링

* 퍼즐 사이즈 입력

In [23]:
size = int(input('-> 퍼즐 사이즈를 입력하세요: '))
print('퍼즐 사이즈: ', size)

-> 퍼즐 사이즈를 입력하세요: 4
퍼즐 사이즈:  4


* 퍼즐 생성

In [24]:
puzzle = initiate_puzzle(size)

* 퍼즐 완성본 생성
 - 기존 퍼즐을 복사하여 생성
 - 아래와 같이 deep copy본으로 생성
   - 그렇지 않으면, 항상 puzzle과 complete가 동일한 객체가 됨

In [25]:
complete = [row[:] for row in puzzle]

* copy 모듈을 이용해서도 가능

In [26]:
import copy
complete = copy.deepcopy(puzzle)

In [27]:
show_puzzle(complete)

  1   2   3   4 
  5   6   7   8 
  9  10  11  12 
 13  14  15     


In [28]:
show_puzzle(puzzle)

  1   2   3   4 
  5   6   7   8 
  9  10  11  12 
 13  14  15     


* 퍼즐 섞기

In [29]:
shuffle_puzzle(puzzle, 300)

In [30]:
show_puzzle(puzzle)

  2   1   9   4 
 12  11   6   8 
  3  13   7  14 
  5  10      15 


#### 게임 루프 
 - 퍼즐이 완성되었나 확인
   - 완성되었다면 종료
   - 완성되지 않았다면 사용자 입력 대기 및 퍼즐 출력 

In [31]:
# output을 clear하기 위해 사용
from IPython.display import clear_output
# 밑줄 부분을 채워봅니다.


show_puzzle(puzzle)
shuffle_puzzle(puzzle,300)
while not is_puzzle_completed(puzzle,complete):
    # 숫자를 입력하지 않은 경우에 대한 예외 처리
    try:
        choice=input(' -> 움직일 숫자를 입력하세요(q를 입력하면 종료) : ')
        if choice=='q': break
        num = int(choice)
    except:
        print('숫자가 아닙니다.')
        continue

    # 움직일 수 선택하기
    move_by_number(puzzle,num)

    # 화면 clear
    clear_output()

    # 움직인 이후 퍼즐 상태 보기
    show_puzzle(puzzle)

# 루프의 종료는 곧 퍼즐의 완성을 의미!
if(choice=='q'):
    print('\n게임종료!')
else:
    print('\n퍼즐 완성!')

  2   1   9   4 
 12  11   6   8 
  3  13   7  14 
  5  10      15 
 -> 움직일 숫자를 입력하세요(q를 입력하면 종료) : ㅂ
숫자가 아닙니다.
 -> 움직일 숫자를 입력하세요(q를 입력하면 종료) : q

게임종료!


---
>- 리뷰
   - 어떤 이유로 함수가 쓰여졌으며 어떻게 써야하는지 잘 이해하셨습니다.
   - 빈칸에 들어갈 함수와 argument를 적절히 쓰셨고요!
   - 참고사항이지만 위와 같이 적절한 함수를 호출하기 위해선 네이밍도 중요합니다.
     - 네이밍에서 함수의 역할을 알지 못하면 매번 그 함수가 무슨 일을 하는지 체크해야 하거든요
   - 추가로 넣어주신 code는 user friendly하게 넣어주셔서 아주 좋았습니다!